In [4]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from sentence_transformers import SentenceTransformer, models, util
import pandas as pd
import os

def createEmbedder():
    #embedding_method = "sentence-transformers/bert-base-nli-mean-tokens"
    #embedding_method = "gsarti/biobert-nli"
    embedding_method = "gsarti/scibert-nli"
    
    #embedder = SentenceTransformer(embedding_method)

    word_embedding_model = models.Transformer(embedding_method, max_seq_length=512, do_lower_case=True)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())    
    return SentenceTransformer(modules=[word_embedding_model, pooling_model])


In [6]:
inputFile = 'Questions_Generations_Annotations.csv'
df = pd.read_csv(inputFile, encoding= 'unicode_escape', index_col=False, names=['ID', 'Sample','Question','Question_Type','Question_Type1','Correct_Grammar','Suggestions'])
df

,ID,Sample,Question,Question_Type,Question_Type1,Correct_Grammar,Suggestions
0,NaN,Sample,Question,Question Type,Question Type,Correct Grammar,Suggestions
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ID,Question / Sentence Samples,1 - Question \n0 - Not a Question,What / When / Why / How / Where / Who / Which ...,Second Label,1 - Correct Grammar\n0 - Incorrect Grammar,Correct grammar question / sentence if it is i...
3,AAG,Do you have to remove tartar from dogÃ¢â¬â¢s...,1,YN,NaN,1,NaN
4,AAJ,I recommend you look for enzymatic toothpaste ...,0,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...
5286,ZZY,Where does we buyer really sad sympathy cards,1,Where,NaN,0,Where do we buy really sad sympathy cards?
5287,ZZR,What animal are at risk?,1,What,NaN,0,Which animals are at risk?
5288,ZZS,What are poisonous to a dogs,1,What,NaN,0,What is poisonous to dogs?
5289,ZZW,It cause high blood sugars in dogs,0,NaN,NaN,0,It causes high blood sugar in dogs.


In [7]:
df.drop(index=df.index[:3], axis=0, inplace=True)

In [8]:
df = df.reset_index()
df

,index,ID,Sample,Question,Question_Type,Question_Type1,Correct_Grammar,Suggestions
0,3,AAG,Do you have to remove tartar from dogÃ¢â¬â¢s...,1,YN,NaN,1,NaN
1,4,AAJ,I recommend you look for enzymatic toothpaste ...,0,NaN,NaN,1,NaN
2,5,AAM,Why do you brush your cat's teeth? What are so...,1,Why,What,1,NaN
3,6,AAS,What was the correct answer to what I heard?,1,What,NaN,1,NaN
4,7,AAU,What was a dog once loved with?,1,What,NaN,1,NaN
...,...,...,...,...,...,...,...,...
5283,5286,ZZY,Where does we buyer really sad sympathy cards,1,Where,NaN,0,Where do we buy really sad sympathy cards?
5284,5287,ZZR,What animal are at risk?,1,What,NaN,0,Which animals are at risk?
5285,5288,ZZS,What are poisonous to a dogs,1,What,NaN,0,What is poisonous to dogs?
5286,5289,ZZW,It cause high blood sugars in dogs,0,NaN,NaN,0,It causes high blood sugar in dogs.


In [9]:
df['Correct_Grammar'].value_counts()

1    4519
0     769
Name: Correct_Grammar, dtype: int64

In [10]:
training_data = df[['Sample', 'Correct_Grammar']]

In [11]:
training_data.head()

,Sample,Correct_Grammar
0,Do you have to remove tartar from dogÃ¢â¬â¢s...,1
1,I recommend you look for enzymatic toothpaste ...,1
2,Why do you brush your cat's teeth? What are so...,1
3,What was the correct answer to what I heard?,1
4,What was a dog once loved with?,1


In [12]:
#Generating embedding for each question
embedder = createEmbedder()
training_data['embeddings'] = training_data.Sample.apply(lambda x: embedder.encode(x))

# #Writing to file
# outputFile = inputFile.replace('.csv','_embed.csv')
# training_data.embeddings.to_csv(outputFile, index=False)

<ipython-input-12-7d8ebcc02043>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data['embeddings'] = training_data.Sample.apply(lambda x: embedder.encode(x))


In [13]:
training_data.head()

,Sample,Correct_Grammar,embeddings
0,Do you have to remove tartar from dogÃ¢â¬â¢s...,1,"[-0.86347216, 1.2074538, 0.5455054, 0.76290923..."
1,I recommend you look for enzymatic toothpaste ...,1,"[0.31379858, 1.3157421, -0.672361, 1.1914873, ..."
2,Why do you brush your cat's teeth? What are so...,1,"[0.6325648, 1.8608861, -0.6300366, 1.1320021, ..."
3,What was the correct answer to what I heard?,1,"[-1.4644822, 1.6326423, -0.024723262, 0.587996..."
4,What was a dog once loved with?,1,"[-0.96746576, 1.7015967, -0.13893355, 0.185108..."


SMOTE

In [14]:
from imblearn.over_sampling import SMOTE

In [15]:
sm = SMOTE(random_state=42)
X_train_smote, y_train_smote = sm.fit_resample(list(training_data.embeddings), training_data.Correct_Grammar.astype('int'))

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_smote, y_train_smote, test_size=0.33, random_state=42)

In [17]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier()
sgd.fit(list(X_train), y_train)

SGDClassifier()

In [18]:
y_pred_sgd = sgd.predict(list(X_test))

In [19]:
from sklearn.linear_model import LogisticRegression


lr = LogisticRegression(max_iter = 1000)
lr.fit(list(X_train), y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [20]:
y_pred_lr = lr.predict(list(X_test))

In [21]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(list(X_train), y_train)

RandomForestClassifier()

In [22]:
y_pred_rf = rf.predict(list(X_test))

In [23]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

nb_gaussian = GaussianNB()
nb_gaussian.fit(list(X_train), y_train)

nb_bern = BernoulliNB()
nb_bern.fit(list(X_train), y_train)

BernoulliNB()

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

pipe = Pipeline([
    ["scale",MinMaxScaler()],
    ["clf",MultinomialNB()]
])

pipe.fit(list(X_train), y_train)

Pipeline(steps=[('scale', MinMaxScaler()), ['clf', MultinomialNB()]])

In [25]:
y_pred_nbg = nb_gaussian.predict(list(X_test))

In [26]:
y_pred_nbb = nb_bern.predict(list(X_test))

In [27]:
y_pred_pipe = pipe.predict(list(X_test))

In [28]:
from sklearn.metrics import confusion_matrix, f1_score

In [29]:
confusion_matrix(y_true=y_test, y_pred=y_pred_sgd)

array([[1213,  255],
       [ 507, 1008]])

In [30]:
confusion_matrix(y_true=y_test, y_pred=y_pred_lr)

array([[1190,  278],
       [ 419, 1096]])

In [31]:
confusion_matrix(y_true=y_test, y_pred=y_pred_rf)

array([[1372,   96],
       [ 129, 1386]])

In [32]:
confusion_matrix(y_true=y_test, y_pred=y_pred_nbg)

array([[923, 545],
       [521, 994]])

In [33]:
confusion_matrix(y_true=y_test, y_pred=y_pred_nbb)

array([[840, 628],
       [598, 917]])

In [34]:
confusion_matrix(y_true=y_test, y_pred=y_pred_pipe)

array([[870, 598],
       [650, 865]])

In [35]:
f1_score(y_true=y_test, y_pred=y_pred_sgd)

0.7257019438444925

In [36]:
f1_score(y_true=y_test, y_pred=y_pred_lr)

0.7587400484596747

In [37]:
f1_score(y_true=y_test, y_pred=y_pred_rf)

0.9249249249249248

In [38]:
f1_score(y_true=y_test, y_pred=y_pred_nbg)

0.6509495743287492

In [39]:
f1_score(y_true=y_test, y_pred=y_pred_nbb)

0.5993464052287582

In [40]:
f1_score(y_true=y_test, y_pred=y_pred_pipe)

0.5809267965077234

In [41]:
sent = "What is the accelerated form of udysiufa decomposition that uses a sterile water solution"

sent_emb = embedder.encode(sent)

In [42]:
wr_sent = "What is the accelerated form of udysiufa decomposition that uses a sterile water solution?"

wr_emb = embedder.encode(wr_sent)

sgd.predict([wr_emb, sent_emb])

array([0, 0])

In [43]:
lr.predict([wr_emb, sent_emb])

array([0, 0])

In [44]:
rf.predict([wr_emb, sent_emb])

array([1, 1])

In [45]:
nb_gaussian.predict([wr_emb, sent_emb])

array([1, 1])

In [46]:
nb_bern.predict([wr_emb, sent_emb])

array([0, 1])

In [47]:
pipe.predict([wr_emb, sent_emb])

array([0, 0])

In [54]:
import joblib

joblib.dump(rf, "./random_forest_grammar_check.joblib")

['./random_forest_grammar_check.joblib']

In [48]:
# import pandas as pd
# import numpy as np
# import seaborn as sns
# from pprint import pprint

# from sklearn.tree import DecisionTreeClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier

# from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, log_loss

# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
# from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

In [49]:
# grid_model_dict = {
#     "logistic_regression": {
#         "model": LogisticRegression,
#         "grid_hyperparams": {
#             "penalty": ["l2", "none"],   # penalty 'l1' and 'elasticnet' is not available for all solvers
#             "C": list(np.arange(0.1, 1, 0.1)),
#             "solver": ["newton-cg", "lbfgs", "sag", "saga"],
#             "max_iter": [50, 75, 100, 150, 200, 300],
#         }
#     },
#     "svc": {
#         "model": SVC,
#         "grid_hyperparams": {
#             "C": list(np.arange(0.1, 1, 0.1)),
#             "kernel": ["linear", "rbf"],
#             "degree": list(range(30, 100, 10)),
#             "gamma": ["scale", "auto"],
#             "max_iter": list(range(80, 500, 50))
#         }
#     },
#     "random_forest": {
#         "model": RandomForestClassifier,
#         "grid_hyperparams": {
#             "n_estimators":  list(range(300, 800, 50)),
#             "criterion": ["gini", "entropy", "log_loss"],
#             "max_depth": list(range(65, 100, 5)),
#             "max_leaf_nodes": list(range(70, 80, 2)),
#             "max_features": [6, 10] + ["log2"],    # 'auto' is deprecated in latest sklearn
#             "bootstrap": [True, False]
#         }
#     },
# }

In [50]:
# grid_score = []
# best_estimator = {}

In [51]:
# for model_key, model_val in grid_model_dict.items():
#     print("*"*10, f"Fitting for {model_key}", "*"*10, "\n")
#     if model_key != "knn":
#         model = model_val["model"](random_state=42)
#     else:
#         model = model_val["model"]()
    
#     grid = GridSearchCV(
#         estimator=model,
#         param_grid=model_val["grid_hyperparams"],
#         verbose=5,
#         n_jobs=-2  # Use all CPU core but 1
#     )
#     grid.fit(list(X_train), y_train)
#     grid_output = grid.cv_results_
#     best_estimator[model_key] = grid.best_estimator_
    
#     for el in zip(grid_output["mean_test_score"], grid_output["params"], grid_output["mean_fit_time"]):
#         grid_score.append({
#             "model": model_key,
#             "accuracy_score": el[0],
#             "params": el[1],
#             "fit_time": el[2]
#         })
        
#     print("\n")
    

In [52]:
# grid_score_df = pd.DataFrame.from_dict(grid_score).sort_values("accuracy_score", ascending=False)
# grid_score_df

In [53]:
# for model_key, model_df in grid_score_df.groupby("model"):
#     print("*"*10, f"Exporting DF for {model_key}", "*"*10, "\n")
#     sampled_df = model_df.sort_values("accuracy_score", ascending=False)
#     sampled_df = sampled_df.drop("model", axis=1)
#     sampled_df.to_excel(f"{model_key}_gridcv.xlsx", index=False)